In [ ]:
import os
import numpy as np
import pandas as pd

from exp_spec_info import *
from plot_info import *
from select_data import *

In [ ]:
# Data pickle path
extracted_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\extracted_data.pkl"

# Processed data paths
extended_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\extended_data.pkl"
median_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\median_data.pkl"

##### Extend Data with Processed Elements

In [ ]:
data = pd.read_pickle(extracted_data_path)
rel_time_ext = []
rel_res_ext = []
rel_res_frac_err_ext = []

# Extend with relative residual
for setup in SETUPS:
    for matrix in SETUP_MATRIX_MAPPING[setup]:
        for restart_param in RESTART_PARAMS:
            for exp_iter in range(N_EXPERIMENT_ITERATIONS):

                sub_data = df_sel_setup_matrix_restart(
                    data, setup, matrix, restart_param
                )
                sub_data = sub_data[sub_data["experiment_iter"] == exp_iter]

                rel_res = sub_data["inner_relres"].apply(
                    lambda relres_arr: relres_arr[-1]
                )
                if len(rel_res) > 0:
                    rel_res_ext.append(rel_res)

rel_res_ext = pd.concat(rel_res_ext)
rel_res_ext.name = "final_relres"

extended_data = data.join((rel_res_ext))

for setup in SETUPS:
    for matrix in SETUP_MATRIX_MAPPING[setup]:
        for restart_param in RESTART_PARAMS:
            for exp_iter in range(N_EXPERIMENT_ITERATIONS):

                sub_data = df_sel_setup_matrix_restart(
                    extended_data, setup, matrix, restart_param
                )
                sub_data = sub_data[sub_data["experiment_iter"] == exp_iter]

                if sub_data[sub_data["solver"] == "FP FP64"].size != 0:

                    fp64_time = sub_data[
                        sub_data["solver"] == "FP FP64"
                    ].iloc[0]["elapsed_time_ms"]
                    rel_time = sub_data["elapsed_time_ms"]/fp64_time
                    rel_time_ext.append(rel_time)

                    # For comparitive analysis regulate all final relres below
                    # minimal relres tol as REL_RES_TOL
                    sub_data.loc[sub_data["final_relres"] <= REL_RES_TOL, "final_relres"] = REL_RES_TOL

                    fp64_relres = sub_data[
                        sub_data["solver"] == "FP FP64"
                    ].iloc[0]["final_relres"]
                    rel_res_frac_err = (sub_data["final_relres"]-fp64_relres)/fp64_relres
                    rel_res_frac_err_ext.append(rel_res_frac_err)

rel_time_ext = pd.concat(rel_time_ext)
rel_time_ext.name = "rel_time"

rel_res_frac_err_ext = pd.concat(rel_res_frac_err_ext)
rel_res_frac_err_ext.name = "rel_res_frac_err"

extended_data = extended_data.join((rel_time_ext, rel_res_frac_err_ext))
display(extended_data)

extended_data.to_pickle(extended_data_path)

##### Compress into Experiment Iteration Median Values

In [ ]:
median_data = []

for setup in SETUPS:
    for matrix in SETUP_MATRIX_MAPPING[setup]:
        for restart_param in RESTART_PARAMS:

            sub_data = df_sel_setup_matrix_restart(
                extended_data, setup, matrix, restart_param
            )

            for solver in SOLVERS:

                solve_experiments = sub_data[sub_data["solver"] == solver]

                if len(solve_experiments) > 0:

                    median_experiment = {
                        "setup": setup,
                        "matrix": matrix,
                        "solver": solver,
                        "restart_param": restart_param,
                        "all_populated": bool(solve_experiments["populated"].all()),
                        "all_initiated": bool(solve_experiments["initiated"].all()),
                        "all_converged": bool(solve_experiments["converged"].all()),
                        "all_terminated": bool(solve_experiments["terminated"].all()),
                        "med_outer_iter": int(solve_experiments["outer_iters"].median()),
                        "med_inner_iter": int(solve_experiments["inner_iters"].median()),
                        "med_time_ms": int(solve_experiments["elapsed_time_ms"].median()),
                        "med_rel_time": solve_experiments["rel_time"].median(),
                        "med_rel_res": solve_experiments["final_relres"].median(),
                        "med_rel_res_frac_err": solve_experiments["rel_res_frac_err"].median()
                    }

                    median_data.append(median_experiment)

median_data = pd.DataFrame(median_data)
display(median_data)

median_data.to_pickle(median_data_path)